# Data Science Capstone Project: Bar Crawler

*Note: For the most detail and full, actual code used to pull, clean, analyze, and map data, see the 'Capstone Project' notebook. The purpose of this report is to serve as a higher-level, more easily-communicable description of the problem; data; methodology; results; discussion; and review/conclusions.*

---

## 1. Introduction/Business Problem

Consider one of these scenarios:
* You are the owner of a travel blog, or
* You work for a travel company writing articles, or
* You work for a company doing research into travel and/or the New York City night life, or
* (A different alternative:) You're a prospective bar or night club owner looking to profit off being in an active nightlife area.

Maybe you want to write an article or report on what life was like before COVID-19, or you're prepping some content for all the people planning to travel once it's over. In any case, our goal is to write an "N Places..." article, because who doesn't love a good 12 Places to Take a Really Cool Selfie in NYC article?

What type of list would be best? Well, why not one that facilitates an adventurous social gathering that invovles a core group and the potential to meet lots of people? The answer is clear: Bar Crawl Spots in NYC! Who doesn't love a good bar crawl?

To do this, we want to give some travelers good advice when visiting New York City, but we're not locals, or very well-versed in the night-life. In fact, this writer is from and residing in California. So we want to use data to help us.

**Our Problem: What _bar crawl spots_ are there in New York City that we can recommend to prospective travelers?**

The answer to this will help us write a great article or report with actual maps of the areas and actual places to go to. Our number of clusters *k* will be the "N" bar crawl areas, but the actual value of k will depend on the *quality* of the clusters: do they contain actual bars, and are they close together?; as well as the *quantity*: are regions that could be considered one crawl split up into two, or three? Are several different crawls being lumped into one?

---

## 2. Data

The data necessary for this project is actually quite minimal. We only need three different sources of data, two of which are publicly available through free-to-use APIs.

### Source #1: Foursquare (Venues and their Locations)

In order to discover bar crawl locations, we needed a substantial amount of actual venues (goal was set to be at least 500). In order to get this data, Foursquare's Places API was utilized. The constraints on the venue data used were fairly minimal: 

1. Venues must be tagged as a 'Nightlife' venue according to Foursquare's documentation (see: https://developer.foursquare.com/docs/build-with-foursquare/categories/);

2. Each venue must be unique (based on name or address);

3. Locations must be within New York City (ignore adjacent venues even if they are nearby others within city limits to establish a definite boundary); and

4. All venues must have valid latitude/longitude coordinates.

For a more detailed description of the methods used to pull and clean the data, see **Methodology**.

### Source #2: NYU Spacial Data Repository (Neighborhoods in NYC and their Locations)

Due to a limitation of the Foursquare Places API which will be discussed in further detail in the Methodology section, it became necessary to have many different latitude/longitude coordinates within NYC. This was made possible by a freely available dataset actually used in the "Segmenting and Clustering Neighborhoods in New York City" lab, which comes from the NYU Spacial Data Repository. The data actually used in the lab is from IBM themselves, who very courteously (for the purposes of the lab work) cleaned the data.

Link to NYU Spacial Data Repository page: https://geo.nyu.edu/catalog/nyu_2451_34572

Link to this course's raw, cleaned dataset: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

### Source #3: Folium (Maps of Bar Crawl Spots)

Folium was used to generate maps of all discovered venues; all venues, clustered; and individual clusters of venues. Folium is incredibly easy to use, and has all the features we need for this analysis. Much of the code used to plot the venues onto Folium maps was taken from the "Segmenting and Clustering Neighborhoods in New York City" lab, and modified to fit the needs of this analysis.

Link to Folium's documentation: https://python-visualization.github.io/folium/

---

## 3. Methodology

The methods used in this project can be described in four approximate steps:

1. Data **Pulling**
2. Data **Cleaning**
3. Data **Analysis**
4. Data **Visualization**

The following four subsections will correspond to each of these approximate steps.

### Method Step 1: Data Pulling

As discussed previously, the Foursquare Places API was used to discover Nightlife venues in New York City. The API call required the following parameters:

* Authentication parameters: Client_Id, and Client_Secret
* Version: just a date; 01/21/2021 was used
* Location: either coordinates or name of place; the former were used (see below)
* Query: search query; to be as general as possible, 'bar' was used
* Limit: maximum value is 50, which was used
* CategoryId: not a necessary parameter, but key for our analysis to restrict results to 'Nightlife' venues

An obstacle to pulling all the data needed for the analysis was the limit on results from a single search query to 50. Running a query on 'New York City' as the location would only give 50 results nearby the coordinates of the city, which was around the Manhattan area. In order to overcome this limitation, the NYU Spatial Data Repository dataset was used. In it, all 306 neighborhoods across the 5 boroughs are detailed with each of their respective coordinates.

Queries to the Places API were done for each of the 306 coordinates in the Neighborhoods dataset using the requests library. The results were transformed into json format; turned into a pandas DataFrame; and stored in a list of the resulting DataFrames. The total number of results was just under the expected 306 * 50 = 15,300.

Finally, the list of pandas DataFrames were concatenated into one complete DataFrame of all search results.

### Method Step 2: Data Cleaning

The complete DataFrame contained 16 features, and (as stated) just under 15,300 rows. To meet the set constraints on the venues data (of the Nightlife category; within NYC; unqiue; and with valid coordinates), cleaning was necessary. The following were performed to achieve these contraints:

1. As previously stated, CategoryId was specified in the search queries to ensure each venue was of the 'Nightlife' category.

2. The pandas "drop_duplicates" method was used on the DataFrame of all results, specifically on the names and addresses of all venues. This ensured venues with either the same name or address were dropped. Only the first of the duplicates was kept. Notably, this eliminated chains (different venues sharing the same name at different addresses); an unfortunate consequence, but since so many results were likely to overlap with only small changes in coordinates across NYC, the decision was made to focus on unique (and perhaps more 'local') destinations instead of 'completeness' of data.

3. The NYC Neighborhoods dataset was used to ensure the venues were within NYC. Due to the 'city' feature of the results actually having values spanning borough, neighborhood, or city, the Neighborhoods dataset was turned into a list of the names of all the unique neighborhoods and boroughs; 'New York' and 'New York City' were also added to keep those venues which were properly labeled as the city. Pandas' selection methods and the "isin" method were used to accomplish this.

4. To ensure all venues had valid coordinates, all rows of the resulting DataFrame which had "NA" values for either latitude or longitude were dropped from the DataFrame by using pandas' "dropna" method.

Finally, the cleaned DataFrame was saved locally using pickle. Additionally, because some features were not necessary, a 'slimmed down' version with these columns removed was also saved.

### Method Step 3: Data Analysis

The k-Means machine learning algorithm was used to discover bar crawl destinations. While potentially not the best possible choice, it was chosen for being easy to use, flexible, and especially simple to explain (number of clusters = number of bar crawl destinations). The Scikit-Learn library was used to apply the algorithm to the data.

The process was remarkably simple. First, a separate DataFrame containing only the coodinates of each venue was made which aligned with the original DataFrame based on its index. Next, this data along with a chosen value of k (see below) were passed to the KMeans function from the Scikit-Learn library. This produced an array of Cluster Labels corresponding to the unique indices of the rows. Finally, this array of labels was inserted into a copy of the DataFrame containing the venues in a new 'Cluster Label' category. Since the indices matched, a simple insert was all that was needed to create the final DataFrame.

#### On Picking k, and Error Analysis

A quirk of this project is the lack of error analysis. A common way to discover the 'correct' value of k is to plot accuracy of the model over values of k within a range, and find the 'elbow point' - the value of k at which increases in k no longer yield gains in accuracy. However, this project is unique in that *there is no 'correct' value of k.* The definition of a "bar crawl destination" is not technical or well-defined, at least quantitatively. Therefore, the process of choosing the final value of k was a heuristic, not a quantitative approach. The two criteria used were: (1) k being a "nice, round" number, and (2) the resulting clusters being at least hypothetically walkable.

Therefore, there is no dedicated error analysis section to this report, and no elbow points discovered. The final value of k was decided based on some trial-and-error and gut feeling. Ultimately, the chosen value of k was 100.

### Method Step 4: Data Visualization

As Folium is built to be easy to use and quite lightweight, all that was needed to construct a map of all venues in the dataset in NYC was to assign a point of a uniform color to each coordinate. In order to construct a map of all venues separated into their respective clusters, different colors were generated from Matplotlib's 'cm' and 'colors' modules, and assigned to each cluster.

However, both of these maps were not very easy to read, and therefore not very useful to anyone looking to use this analysis for any of the purposes described at the beginning of this report. Therefore, additional code was written to instead plot a single selected cluster at a time to the map of NYC to analyze one bar crawl destination at a time, and finally generate recommendations.

---

## 4. Results

As stated, just under 15,300 search results were slimmed down to a total of 672, well over the goal of 500. The map of all these venues generated by Folium is shown in Figure 1.

<img src='all_bars_in_nyc.png'>
 Figure 1: All 627 results, ungrouped.

Using k = 100, the clusters discovered by the sklearn k-Means algorithm are shown in Figure 2, each color-coded.

<img src='clustered_bars.png'>
Figure 2: 627 results segmented into 100 clusters.

While the clusters near Manhattan are relatively easy to tell apart, they clearly become thinner the further away from the heart of the city one goes. The high value of k clearly prioritizes the heart of the city, clustering the less-dense outer areas less effectively. Figure 3 shows a zoomed-in view of Manhattan and its surrounding areas.

<img src='clustered_bars_zoomed.png'>
Figure 3: Clusters in and around Manhattan.

---

## 5. Discussion

The choice of k, as discussed in the Methodology section, was generally based on heuristics and gut feeling rather than any kind of quantiative measure. Upon reflecting on the results, it becomes obvious that the analysis ended up being quite Manhattan-centric. Perhaps this should have been expected at outset, but the results have shown it definitively. Therefore, in looking at the results, instead of giving a list of all 100 clusters and calling them all "bar crawl destinations," the data available and analysis may suggest pivoting to being more of a "Bar Crawl Spots in Manhattan" project, and only showing those around and south of Central Park.

The corresponding notebook containing all code used for the project may be referred to to explore the dataset and discovered clusters further; further tweaking could yield great results for even more robust results.

One avenue for improved results could be limiting results to those within Manhattan. This would involve going back through the code and doing queries only on coordinates within Manhattan. Because the 'city' column varied in its resolution, the full data of 15,300 venues as already pulled cannot be slimmed down to only locations in Manhattan without knowledge of what coordinates are precisely within the borough.

A few examples of the bar crawl destinations discovered are shown below. The first three are in southern Manhattan, and the final is in northern Brooklyn.

<img src='cluster_85.png'>
Figure 4: Cluster 85 in southern Manhattan.

<img src='cluster_98.png'>
Figure 5: Cluster 98 in southern Manhattan.

<img src='cluster_35.png'>
Figure 6: Cluster 35 in southern Manhattan.

<img src='cluster_33.png'>
Figure 7: Cluster 33 in northern Brooklyn.

## 6. Conclusion

It is immediately clear from looking at every full-city map generated that NYC nightlife is - by the metric of Foursquare nightlife venue data - concentrated strongly in southern Manhattan and its many neighborhoods. A future version of this analysis could be done on Manhattan in particular, even specifying particular neighborhoods within it, to increase resolution and accuracy.

As it stands now, the current clusters seem too 'thin' and spread out. Ten blocks is definitely doable in a night, if one is willing to sacrifice the next day to it.

However, as a writer or travel agent doing research to make recommendations to potential travelers, or a prospective entrepreneur looking to open a bar or other nightlife-focused venue, this analysis could be very fruitful, especially if the focus is narrowed onto a handful of neighborhoods, or one borough at a time; etc. 

If you are looking for a bar crawl in NYC, it seems from this analysis that southern NYC is your best bet! It has nothing to say about the quality of each location, but if quantity and proximity of venues is important to you, and you just happen to be that rare individual that enjoys both data science and bar crawls, this analysis presents the opportunity to help you plan your next late-night post-pandemic adventure (though this writer admits that those are, generally, best when *un-*planned).